In [9]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import csv
from ast import literal_eval

import spacy
from empath import Empath
import gensim
from gensim import corpora
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, PlaintextCorpusReader
from nltk.stem import WordNetLemmatizer
import nltk
from collections import Counter

from cliches import *

In [2]:
np.random.seed(47)

In [3]:
df_uk["attribute"].to_numpy().tolist() 

NameError: name 'df_uk' is not defined

In [4]:
# load seegull data
df_cliches = pd.read_csv("data/seegull.csv")
# extract attributes related to British and English
df_uk = df_cliches[np.isin(df_cliches["identity"], ["British", "English"])]
attributes_uk = df_uk["attribute"].to_numpy().tolist()
# remove all stopwords
new = []
english_stopwords = stopwords.words("english")
for a in attributes_uk:
    new.extend([word for word in a.split(" ") if word not in english_stopwords])
joined_attributes_uk = " ".join(new)
print("attributes of UK:\n ", joined_attributes_uk)
# find main topics 
print("\nmain topics:")
top_10 = top_k_categories(joined_attributes_uk, 10, True)
uk_topics_seegull = [t[0] for t in top_10]

attributes of UK:
  arrogant arrogant domineer cowardly bad teeth snob conceited snooty bad teeth snobby uncontrolled cold stiff upper lip aristocratic pompous refine tea drinking annoyed tea country theory good swimmer uncertain stiff upper lip good making tea british pompous understated tea queen stuck multiracial stuffy vapid labour trendy fantastic chess fussy multifaith genero self deprecate sophisticated sophisticated stiff imperialist stiff self indulgent proper superior direct proper love tea civilize forward elegant drink tea culture reserve polite good cricket fair intelligent practical brave red word aloof cat parochial generous polite drink tea intelligent love tea superior refine witty hardworking tasteful civilize progress self control reserve unyielding fair honourable practical shy clever bore aloof considerate brave sociable progressive generous warmth courageous expect kind art travel eat lot colour bought outgo affectionate

main topics:
liquid               7.874%
b

In [5]:
# rename the keys in the column "article_name" by removing the suffix ".txt"
# topics_file = "data/topics_articles.csv"
# article_topics = pd.read_csv(topics_file)
# np.set_printoptions(linewidth=100000)
# article_topics["article_name"] = article_topics["article_name"].str.replace(".txt", "")
# article_topics.drop(columns=["Unnamed: 0"], inplace=True)
# article_topics.to_csv(topics_file, index=False)

In [6]:
# topics in each article
topics_file = "data/topics_articles.csv"
article_topics = pd.read_csv(
    topics_file,
    usecols=["article_name", "topics"],
    converters={
        "topics": (lambda s: literal_eval("[" + (",".join(s[1:-1].split(" "))) + "]"))
    },
)
display(article_topics)

,article_name,topics
0,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,"[wedding, family, royalty, medieval, death, he..."
1,%C3%85land,"[beach, water, exotic, sailing, ocean, help, o..."
2,%C3%89douard_Manet,"[art, appearance, fashion, beauty, feminine, c..."
3,%C3%89ire,"[government, help, office, dance, money, weddi..."
4,%C3%93engus_I_of_the_Picts,"[wedding, family, royalty, medieval, death, yo..."
...,...,...
4599,Zionism,"[help, office, dance, money, wedding, domestic..."
4600,Zirconium,"[tool, breaking, help, office, dance, money, w..."
4601,Zoroaster,"[celebration, ancient, meeting, party, help, o..."
4602,Zuid-Gelders,"[speaking, hearing, shape_and_size, help, offi..."


In [7]:
# United Kingdom topics
uk_topics = (
    article_topics[article_topics["article_name"] == "United_Kingdom"]["topics"].values[
        0
    ]
    + article_topics[article_topics["article_name"] == "England"]["topics"].values[0]
    + article_topics[article_topics["article_name"] == "Great_Britain"]["topics"].values[0]
    + uk_topics_seegull
)


# Function to extract common topcis
def extract_common(topics_str1: str, topics_str2: str) -> set:
    return set(topics_str1) & set(topics_str2)


# Compare topics for all articles in the dataset
articles_with_common_topics = []
for index, row in article_topics.iterrows():
    if row["article_name"] not in {"United_Kingdom", "England"}:
        common_topics = extract_common(row["topics"], uk_topics)
        if len(common_topics) > 0:
            articles_with_common_topics.append(
                {"article_name": row["article_name"], "common_topics": common_topics}
            )

# Sort comparisons based on the number of common topics
articles_with_common_topics.sort(key=lambda x: len(x["common_topics"]), reverse=True)

# Print the top 5 comparisons
print("Top 5 comparisons with most common topics:")
for i in range(min(5, len(articles_with_common_topics))):
    print(
        f"{articles_with_common_topics[i]['article_name']} and United Kingdom: {articles_with_common_topics[i]['common_topics']}"
    )

# Save the top 50 comparisons to a CSV file
top_articles_in_common_topics = pd.DataFrame(articles_with_common_topics)
top_articles_in_common_topics.to_csv("data/ted_top_topic_comparison.csv", index=False)

Top 5 comparisons with most common topics:
%C3%85land and United Kingdom: {'help', 'office', 'beach', 'wedding', 'sailing', 'ocean', 'water', 'dance', 'money', 'exotic'}
Achilles and United Kingdom: {'help', 'office', 'beach', 'wedding', 'sailing', 'ocean', 'water', 'dance', 'money', 'exotic'}
Alfred_the_Great and United Kingdom: {'royalty', 'help', 'office', 'wedding', 'medieval', 'positive_emotion', 'achievement', 'power', 'dance', 'money'}
American_Samoa and United Kingdom: {'help', 'office', 'beach', 'wedding', 'sailing', 'ocean', 'water', 'dance', 'money', 'exotic'}
Antigua_and_Barbuda and United Kingdom: {'help', 'office', 'beach', 'wedding', 'sailing', 'ocean', 'water', 'dance', 'money', 'exotic'}


# NOms propres

In [10]:
tokens_and_pos = pd.read_csv("data/preprocessed_data.csv")
tokens_and_pos["Article"] = tokens_and_pos["Article"].str.replace(".txt", "")

In [12]:
# Replace 'United_Kingdom.txt' with the target article name
target_article_name = 'United_Kingdom'

# Filter rows related to the target article
target_article_data = tokens_and_pos[tokens_and_pos['Article'] == target_article_name]

# Check if the target article is present in the DataFrame
if not target_article_data.empty:
    # Extract the top 10 PROPN tokens for the target article
    target_proper_nouns = [token[0] for tokens_pos_list in target_article_data['Tokens_POS'] for token in eval(tokens_pos_list) if token[1] == 'PROPN']
    top_target_proper_nouns = [item[0] for item in Counter(target_proper_nouns).most_common(10)]

    # Initialize a list to store common proper nouns
    common_proper_nouns = []

    # Iterate through each row in the DataFrame
    for index, row in tokens_and_pos.iterrows():
        if row['Article'] == target_article_name:
            continue
        # Extract the top 10 PROPN tokens for each article
        article_proper_nouns = [token[0] for token in eval(row['Tokens_POS']) if token[1] == 'PROPN']
        top_article_proper_nouns = [item[0] for item in Counter(article_proper_nouns).most_common(10)]

        # Compare with the top PROPN tokens of the target article
        common_tokens = set(top_article_proper_nouns) & set(top_target_proper_nouns)

        # Store the results
        common_proper_nouns.append({'Article': row['Article'], 'Common_Propnouns': common_tokens})

    # Convert the results to a DataFrame
    common_proper_nouns_df = pd.DataFrame(common_proper_nouns)

    # Sort the DataFrame based on the number of common proper nouns
    common_proper_nouns_df = common_proper_nouns_df.sort_values(by='Common_Propnouns', key=lambda x: x.str.len(), ascending=False)

    # Print the top 5 comparisons
    print("Top 5 comparisons with most common proper nouns:")
    for i in range(min(5, len(common_proper_nouns_df))):
        print(f"{common_proper_nouns_df.iloc[i]['Article']} and {target_article_name}: {common_proper_nouns_df.iloc[i]['Common_Propnouns']}")

    # Save the top comparisons to a CSV file
    top_comparisons = common_proper_nouns_df.head(50)
    if not top_comparisons.empty:
        top_comparisons.to_csv("data/top_proper_noun_comparisons.csv", index=False)
        print("Top comparisons with most common proper nouns saved to data/top_proper_noun_comparisons.csv")
    else:
        print("No common proper nouns found.")
else:
    print(f"{target_article_name} not found in the DataFrame.")


Top 5 comparisons with most common proper nouns:
United_Kingdom_national_football_team and United_Kingdom: {'Ireland', 'UK', 'Kingdom', 'Scotland', 'Northern', 'United', 'England'}
Pound_sterling and United_Kingdom: {'Ireland', 'UK', 'Kingdom', 'Scotland', 'Northern', 'United', 'England'}
Great_Britain and United_Kingdom: {'Ireland', 'Kingdom', 'Scotland', 'Northern', 'United', 'England', 'British'}
England and United_Kingdom: {'London', 'UK', 'Kingdom', 'Scotland', 'United', 'England'}
British_Isles and United_Kingdom: {'Ireland', 'Kingdom', 'Scotland', 'United', 'England', 'British'}
Top comparisons with most common proper nouns saved to data/top_proper_noun_comparisons.csv


# links

In [13]:
from preprocessing import import_and_clean_data
(
    articles,
    categories,
    links,
    paths_finished,
    paths_unfinished,
) = import_and_clean_data()

In [14]:
def create_set(link, df):
    return set(df[df["linkSource"] == link]["linkTarget"].to_numpy())

# reference article
set1 = create_set("United_Kingdom", links)

# related article ("cliché")
set2 = create_set("William_IV_of_the_United_Kingdom", links)

# unrelated articles
set3 = create_set("Zulu", links)
set4 = create_set("World_Bank_Group", links)

In [15]:
def write_to_csv(data_path_article, df, columns, header=True, index=False):
    if os.path.isfile(data_path_article):
        with open(data_path_article, "a") as csvfile:
            csvwriter = csv.writer(csvfile)

            csvwriter.writerows(df.to_numpy())
    else:
        df.to_csv(
            data_path_article,
            sep=",",
            encoding="utf-8",
            columns=columns,
            header=header,
            index=index,
        )

# create dataframe that contains the links in common between the reference article and an article
# done on all the articles
# /!\ dataset is stored in "links_common.csv" file
df_links_common = pd.DataFrame(
    columns=["reference_article", "article", "common_articles"]
)
reference_article = "United_Kingdom"

reference_set = create_set(reference_article, links)

for article_name in links["linkSource"].unique():
    comparison_set = create_set(article_name, links)
    new_row = {
        "reference_article": reference_article,
        "article": article_name,
        "common_articles": list(reference_set & comparison_set),
    }
    df_links_common.loc[df_links_common.shape[0]] = new_row

write = True
if write:
    write_to_csv("data/links_common.csv", df_links_common, ["reference_article", "article", "common_articles"])

common_links_reference = pd.read_csv("data/links_common.csv", encoding="utf-8")
common_links_reference

,reference_article,article,common_articles
0,United_Kingdom,Áedán_mac_Gabráin,"['Orkney', 'Isle_of_Man', 'Monarchy', 'Great_B..."
1,United_Kingdom,Åland,"['Time_zone', 'World_War_II', 'Currency', 'Eur..."
2,United_Kingdom,Édouard_Manet,"['Germany', 'United_States_dollar', 'Italy', '..."
3,United_Kingdom,Éire,"['English_language', 'Ireland', 'Republic_of_I..."
4,United_Kingdom,Óengus_I_of_the_Picts,"['Lion', 'Great_Britain', 'Ireland', 'Scotland..."
...,...,...,...
9169,United_Kingdom,Zionism,"['Argentina', 'Germany', 'United_Nations', 'Br..."
9170,United_Kingdom,Zirconium,"['India', 'Bicycle', 'Electron', 'Steel']"
9171,United_Kingdom,Zoroaster,"['India', 'Christianity']"
9172,United_Kingdom,Zuid-Gelders,[]


In [17]:
import ast

# forgotten cliche: Driving_on_the_left_or_right

article_topics = pd.read_csv(
    "data/links_common.csv",
    usecols=["article_name", "topics"],
    converters={
        "topics": (lambda s: literal_eval("[" + (",".join(s[1:-1].split(" "))) + "]"))
    },
)

# get most in common links
common_links_reference["nbr_common_articles"] = common_links_reference["common_articles"].apply(lambda common_articles: len(ast.literal_eval(common_articles)))
common_links_reference.sort_values("nbr_common_articles", ascending=False)[:31]

,reference_article,article,common_articles,nbr_common_articles
8866,United_Kingdom,United_Kingdom,"['Anglican_Communion', 'Bicycle', 'Ireland', '...",168
4279,United_Kingdom,United_Kingdom,"['Natural_gas', 'Arctic_Monkeys', 'Novel', 'Qu...",168
5966,United_Kingdom,England,"['French_language', 'Scots_language', 'London'...",63
1379,United_Kingdom,England,"['Industrial_Revolution', 'Benjamin_Britten', ...",63
3642,United_Kingdom,Scotland,"['Industrial_Revolution', 'Parliament_of_the_U...",57
8229,United_Kingdom,Scotland,"['French_language', 'Scots_language', 'London'...",57
4283,United_Kingdom,United_States,"['Natural_gas', 'World_War_II', 'NATO', 'World...",42
8870,United_Kingdom,United_States,"['Computer', 'French_language', 'Jersey', 'NAT...",42
5287,United_Kingdom,British_monarchy,"['French_language', 'Anglican_Communion', 'Ire...",35
700,United_Kingdom,British_monarchy,"['Parliament_of_the_United_Kingdom', 'World_Wa...",35
